In [2]:
with open('y.txt', 'r') as file:
    data = file.read()


In [3]:
!pip install datasets
!pip install tensorflow
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install transformers
import datasets
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering
!pip install tensorflow-transform


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 21.4 MB/s eta 0:00:00
  Using cached dill-0.3.8-py3-none-any.whl (116 kB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.54.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.54.0 requires pyarrow<15.0.0,>=3.0.0, but you have pyarrow 15.0.1 which is incompatible.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.1 which is incompatible.
tensorflow-transform 1.14.0 requires pyarrow<11,>=10, but you have pyarrow 15.0.1 whic

In [11]:
import json

# Assuming `data` is your text content
json_data = json.loads(data)


In [12]:
with open('y.json', 'w') as json_file:
    json.dump(json_data, json_file, indent=4)


In [13]:
import json

def load_dataset(file_path):
    with open(file_path, 'r') as file:
        dataset = json.load(file)
    return dataset

dataset = load_dataset('y.json')



In [7]:
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)


In [8]:
from transformers import RobertaTokenizerFast
import torch

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

def tokenize_dataset(dataset, tokenizer):
    tokenized_dataset = []
    for item in dataset:
        question = item['question']
        context = item['context']

        # Tokenize the question and context
        inputs = tokenizer(question + context, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

        tokenized_dataset.append(inputs)
    return tokenized_dataset

train_tokenized_dataset = tokenize_dataset(train_dataset, tokenizer)
test_tokenized_dataset = tokenize_dataset(test_dataset, tokenizer)


In [9]:
from transformers import RobertaForQuestionAnswering
from torch.utils.data import DataLoader

# Initialize the model
model = RobertaForQuestionAnswering.from_pretrained('roberta-base').to('cuda')


# Setup the DataLoader for your tokenized dataset
train_dataloader = DataLoader(train_tokenized_dataset, batch_size=8, shuffle=True)

# Setup your training loop here, including defining your loss function and optimizer


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# Example question
question = "Does sound travel faster in space?"

# Tokenize the question
inputs = tokenizer(question, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

# Move inputs to the device where the model is
inputs = {k: v.to('cuda') for k, v in inputs.items()}

# Get model predictions
outputs = model(**inputs)

# Interpret the output
start_scores, end_scores = outputs.start_logits, outputs.end_logits
start_index = torch.argmax(start_scores)
end_index = torch.argmax(end_scores)

# Ensure the indices are within the range of the input_ids tensor
if start_index >= len(inputs["input_ids"][0]) or end_index >= len(inputs["input_ids"][0]):
    print("Error: Start or end index out of range")
else:
    # Convert the token IDs back to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1])

    # Convert tokens to text and print the answer
    answer = tokenizer.convert_tokens_to_string(tokens)
    print(answer)


 in space?
